# Big Data Modeling and Management 2021


## 🚚 BDMM Third Homework Assignment 🚚 

_The Wide World Importers (WWI) is a wholesales novelty goods importer and distributor operating from the San Francisco bay area. In this assignment we will be working with their database._ 
You can get more information and details about the WWI database can be found in the following link: https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-what-is?view=sql-server-ver15

The focus of the third assignment is modelling. We will use the same data source that was used the previous assignment, the World Wide Importers database, and convert it to a document-based database. To that end, we will be  leveraging concepts like data denormalization, indexes, and mongodb design patterns. 

More information on the extended datamodel to be found here: </br>  
https://docs.microsoft.com/en-us/sql/samples/wide-world-importers-oltp-database-catalog?view=sql-server-ver15

## Problem Description

Your team has just arrived at WWI (a leading company in logitics). Welcome!   <br>
Even though business is striving, the IT department is going through a bad time.   <br>
Digitalization was never a priority for the company and now the company operational and analytical requirements is starting to grow beyond the capabilities of their existing data architecture.   <br>

WWI data is spread accross different systems. Namely, an old SQL database, data extracted through an API, and data stored in CSV files. <br>
Currently, the costs to develop the necessary queries to collect data to answer questions asked by the different departments are too high. <br>
Management concluded it is the right time to revise and revamp the data architecture, in order to speed up operations. 

In that context, your team was tasked with merging all the company data into a single and coherent Mongo database. <br>
It is expected that, with your solution, WWI will have a better understanding of their business and that the different departments will be able to obtain efficiently the answers they desperatly need.

The WWI team shared with you an ERD of their current datamodel:<br>
![datamodel](WWI.png)

Addtionally, the WWI team asked you the deliver the following outputs in **10 days**:
- Understand and model the database.  
- Migrate all data to the database
- Answer the questions.  
- Submit the results by following the instructions.  
- Prepare a short oral presentation to explain your design choices and the results you obtained.

With these deliveries, you will have created a prototype and allows the management to decide whether MongoDB is a good solution that meets their requirements.

### Design Requirements

You have been informed that the WWI has the following query requirements to the database.

The web team needs:  
- From which state province are our suppliers from?   
- From which state province are the customers who have a higher credit limit?  


The warehouse group needs:  
- To know which items get ordered together the most?   
- Which items get ordered the most in bulk (bigger amounts)?  
- Which customers have delivery addresses under 10km of distance?  

The CFO:  
- Would like to know the monthly order count?  
- Would like to know the average monthly sales prices?  
- Would like to know the yearly expenditures with suppliers (per supplier name)?  

Partnerships:  
- Would like to know what's the most common payment type?  
- Which supplier of `Novelty Goods Supplier` as the most transactions?  

The marketing team:  
- Want to make an appreciation post and needs the name of the sales person with the most invoices in 2013 (person who's customers brought the most money)?

---

Transform the SQL tables, API results and CSV files provided in the annex with this file and model a database following mongo's best practises.

Write MongoDB queries to awnser the above mentioned queries

Take advantage of database indexes to improve your query speeds

### Deliverables

1. Notebook with all DB creation operations and CRUD operations;
2. Second notebook with all required 'queries to

### Data Source Materials

For the development of this assignment you will have access to the RDBMS/SQL database hosting the original WWI database. To connect to the database use the following credentials:
```
host:rhea.isegi.unl.pt
user:wwi-read-only-user
pass:jGp2GCqrss6nfTEu5ZawhW3mksLsQYQb
database:WWI

# !pip install mysql-connector-python
import mysql.connector
mydb = mysql.connector.connect(host={host}, user={user}, database={database}, port=3306, password={password})
mycursor = mydb.cursor()

mycursor.execute('SHOW TABLES;')
print(f"Tables: {mycursor.fetchall()}")
mycursor.execute('DESCRIBE Purchasing_PurchaseOrderLines;')
print(f"Purchasing_PurchaseOrderLines schema: {mycursor.fetchall()}")
```

Additionally you have access to the following documents.

CSV with Warehouse Data  
**https://liveeduisegiunl-my.sharepoint.com/:f:/g/personal/fpinheiro_novaims_unl_pt/Eh8Mj-m6r4dOt84tPDGUnhUBd5oMC0CJKAeyJm3urNB-8g?e=JuPMuW**

API with Application data  
**http://rhea.isegi.unl.pt:8080/**

## Additional Information

#### Groups  

This is a group activity. <br>
Students should form groups of at least 4 and at most 5. <br>
We will use the current defined groups that have been established during the previous assignments, and that are identified on Moodle.

#### MongoDB database access  

Each group will have access to its own mongodb instance.<br>
Each group will receive an email with their access credentials. <br>
You will use the database to store your results. <br>

Connection details will have the following template:<br>
```
Host: rhea.isegi.unl.pt:27017  
Username: {groups_username}  
Password: {groups_password}  
```
Which then can be used as follows:
```
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}/")
```

#### Submission  Deadline

The submission must contain a notebook with the queries and their results, also indicate the name of the database that you created. <br>
Upload the notebook on moodle before **23:59 of May 30nd**

#### Evaluation   

The third homework assignment counts 20% towards your final mark of the curricular unit. <br>
The assignment will be scored from 0 to 20. <br>
Your final task will be to present the owner of the company your database proposal and how would it make everyone satisfied. <br>

Each group submission will be evaluated on two components:
1. correctness of results;
2. simplicity of the solution;

50% -  Database design  
50% -  Query results  
*    25% - Correctness of queries   
*    25% - Right results

Please note that all code delivered in this assignment will go through plagiarism automated checks. <br>
Groups high similarity levels in their code will undergo investigation.

**Presentations**

Presentations will be held between the 2nd and 3rd of June and you need to sign up your group in this calendly link:<br>
https://calendly.com/d/m9sj-qwpk/presentations (Please try to avoid empty windows)

## Imports

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
from pymongo import MongoClient

## Connect to mongo database

In [2]:
# Connect to Mongo server
host="rhea.isegi.unl.pt"
port="27049"
user="GROUP_32"
password="bRG2XjRZhrRA9IfpmENyXxMlWQDUJdzL"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Connect to mongo db
db = client.denormalised

In [3]:
# List collections of this db
db.list_collection_names()

['Application_Countries',
 'Warehouse_Colors',
 'Sales_OrderLines',
 'Warehouse_StockItemTransactions',
 'Application_StateProvinces',
 'Sales_Invoices',
 'Sales_Orders',
 'Purchasing_PurchaseOrders',
 'customers_payments',
 'Application_DeliveryMethods',
 'Application_Cities',
 'Application_TransactionTypes',
 'Warehouse_StockItems',
 'Sales_InvoiceLines',
 'Warehouse_StockItemStockGroups',
 'Purchasing_PurchaseOrderLines',
 'Purchasing_SupplierCategories',
 'Application_PaymentMethods',
 'Sales_Customers',
 'Sales_CustomerCategories',
 'Purchasing_SupplierTransactions',
 'Application_People',
 'Warehouse_PackageTypes',
 'Purchasing_Suppliers',
 'reviews',
 'Warehouse_StockGroups',
 'Sales_CustomerTransactions']

The web team needs:  
- From which state province are our suppliers from?   


In [4]:
query_1 = {
    '$project':{
                'supplier':'$SupplierName',
                'state_province':'$DeliveryCity.StateProvince.StateProvinceName'
    }
}



pipeline = [query_1]

r = db.Purchasing_Suppliers.aggregate(pipeline)

result = list(r)

result

[{'_id': 1, 'supplier': 'A Datum Corporation', 'state_province': 'Indiana'},
 {'_id': 2, 'supplier': 'Contoso, Ltd.', 'state_province': 'Washington'},
 {'_id': 3,
  'supplier': 'Consolidated Messenger',
  'state_province': 'California'},
 {'_id': 4, 'supplier': 'Fabrikam, Inc.', 'state_province': 'Kentucky'},
 {'_id': 5,
  'supplier': 'Graphic Design Institute',
  'state_province': 'Missouri'},
 {'_id': 6, 'supplier': 'Humongous Insurance', 'state_province': 'Tennessee'},
 {'_id': 7, 'supplier': 'Litware, Inc.', 'state_province': 'California'},
 {'_id': 8, 'supplier': 'Lucerne Publishing', 'state_province': 'Tennessee'},
 {'_id': 9, 'supplier': 'Nod Publishers', 'state_province': 'North Carolina'},
 {'_id': 10,
  'supplier': 'Northwind Electric Cars',
  'state_province': 'New Jersey'},
 {'_id': 11, 'supplier': 'Trey Research', 'state_province': 'South Dakota'},
 {'_id': 12, 'supplier': 'The Phone Company', 'state_province': 'Minnesota'},
 {'_id': 13, 'supplier': 'Woodgrove Bank', 'stat

- From which state province are the customers who have a higher credit limit? 

In [5]:
db.Sales_Customers.create_index(
    [('CreditLimit', 1)],
    name='CreditLimit', 
)

'CreditLimit'

In [6]:
#top 5 creditlimit customers's state provinces
query_1 = {
    '$match':{  'CreditLimit':  {"$exists": True}  }
}


query_2 = {'$project':
          {'credit_limit':'$CreditLimit',
          'state_province':'$DeliveryCity.StateProvince.StateProvinceName'}}

query_3 = {'$sort':
          {'credit_limit':-1}}

query_4 = {'$limit':5}


pipeline = [query_1, query_2, query_3, query_4]

r = db.Sales_Customers.aggregate(pipeline)

result = list(r)

result

[{'_id': 492, 'credit_limit': 4630.5, 'state_province': 'Texas'},
 {'_id': 170, 'credit_limit': 4200.0, 'state_province': 'Tennessee'},
 {'_id': 605, 'credit_limit': 4095.0, 'state_province': 'North Carolina'},
 {'_id': 500, 'credit_limit': 4095.0, 'state_province': 'Wyoming'},
 {'_id': 613, 'credit_limit': 4000.0, 'state_province': 'Oregon'}]

In [7]:
#top 5 state provinces with higher average credit limit
query_1 = {
    '$match':{  'CreditLimit':  {"$exists": True}  }
}


query_2 = {'$project':
          {'credit_limit':'$CreditLimit',
          'state_province':'$DeliveryCity.StateProvince.StateProvinceName'}}

query_3 = {'$group':
          {'_id':'$state_province',
          'avg_creditLimit':{'$avg':'$credit_limit'}}}


query_4 = {'$sort':
          {'avg_creditLimit':-1}}

query_5 = {'$limit':5}


pipeline = [query_1, query_2, query_3, query_4, query_5]

r = db.Sales_Customers.aggregate(pipeline)

result = list(r)

result

[{'_id': 'Tennessee', 'avg_creditLimit': 3157.5},
 {'_id': 'Arkansas', 'avg_creditLimit': 2883.125},
 {'_id': 'Indiana', 'avg_creditLimit': 2832.9166666666665},
 {'_id': 'Oregon', 'avg_creditLimit': 2823.75},
 {'_id': 'Wyoming', 'avg_creditLimit': 2799.1666666666665}]

The warehouse group needs:  
- To know which items get ordered together the most?   




In [8]:
db.Sales_OrderLines.create_index(
    [('OrderID', 1)],
    name='OrderID', 
)

'OrderID'

In [9]:
db.Sales_OrderLines.create_index(
    [('Description', 1)],
    name='Description', 
)

'Description'

In [10]:
query_1 =     {
        '$lookup': {
           "from": "Sales_OrderLines",
           "localField": "OrderID",
           "foreignField": "OrderID",
           "as": "joint_orders"
        }}



query_2 = {
    '$unwind': '$joint_orders'
}

query_3 = {
    '$unwind': '$joint_orders.Description'
}
    




query_4 = {'$project':
          {'_id':False,
          'ol1':'$Description',
          'ol2':'$joint_orders.Description',
           "Eq": {"$eq":["$Description","$joint_orders.Description"]}
            }}

query_5 = {'$match':
            {"Eq": False}}


query_6 = {'$group':
          {'_id': 
           {'ol1':'$ol1',
            'ol2':'$ol2'   },
           'count': { '$sum': 1 }}}

query_7 = {'$sort':{'count':-1}}

query_8 = {'$limit': 30}


pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7, query_8]

r = db.Sales_OrderLines.aggregate(pipeline)

result = list(r)

result

[{'_id': {'ol1': 'DBA joke mug - I will get you in order (White)',
   'ol2': 'Animal with big feet slippers (Brown) M'},
  'count': 15},
 {'_id': {'ol1': 'Animal with big feet slippers (Brown) M',
   'ol2': 'DBA joke mug - I will get you in order (White)'},
  'count': 15},
 {'_id': {'ol1': 'Black and orange handle with care despatch tape  48mmx75m',
   'ol2': 'DBA joke mug - two types of DBAs (Black)'},
  'count': 14},
 {'_id': {'ol1': 'DBA joke mug - two types of DBAs (Black)',
   'ol2': 'Black and orange handle with care despatch tape  48mmx75m'},
  'count': 14},
 {'_id': {'ol1': 'DBA joke mug - you might be a DBA if (White)',
   'ol2': 'RC vintage American toy coupe with remote control (Black) 1/50 scale'},
  'count': 13},
 {'_id': {'ol1': '10 mm Anti static bubble wrap (Blue) 10m',
   'ol2': 'Dinosaur battery-powered slippers (Green) M'},
  'count': 13},
 {'_id': {'ol1': 'Superhero action jacket (Blue) 4XL',
   'ol2': 'IT joke mug - that behavior is by design (White)'},
  'count': 

- Which items get ordered the most in bulk (bigger amounts)?  


In [11]:
query_1={
    '$group': {
        '_id': {'Description' : '$Description'},
        'average_quantity' : {'$avg' : '$Quantity'}
    }
}

query_2 = {
    '$sort' : { 'average_quantity' : -1 }
}

query_3 = {
    '$limit': 3}

query_4 = {
    '$project': {
        '_id' : False,
        'name' : '$_id.Description',
        'Quantity': '$average_quantity'}
}

pipeline = [query_1, query_2, query_3,query_4]

r = db.Sales_OrderLines.aggregate(pipeline)

result = list(r)

result

[{'name': 'Black and orange fragile despatch tape 48mmx75m',
  'Quantity': 198.8181818181818},
 {'name': 'Black and orange fragile despatch tape 48mmx100m',
  'Quantity': 197.26182965299685},
 {'name': 'Shipping carton (Brown) 356x356x279mm',
  'Quantity': 146.47058823529412}]

-Which customers have delivery addresses under 10km of distance to the customer with customerID 961?

In [ ]:
query_1={
    '$project': {
        '_id': False,
        'CostumerID' : '$CustomerID',
        'location' : {'lat':'$DeliveryLocationLat', 'long':'$DeliveryLocationLong' }
    }
}

query_2={
    '$find': 
   {
     '$location':
       { '$near':
          {
            '$geometry': {'location': [ 41.4972, -102.62 ] },
            '$minDistance': 0,
            '$maxDistance': 10000
          }
       }
   }
}
pipeline = [query_1,query_2]

r = db.Sales_Customers.aggregate(pipeline)


result = list(r)

result

The CFO:  
- Would like to know the monthly order count?  


In [12]:
# According to this link: https://docs.mongodb.com/v5.0/core/aggregation-pipeline/#:~:text=Pipeline%20Operators%20and%20Indexes,
# this query is not possible to be done with an index, because of the accumulator

query_1 = {
    '$group': {
        '_id' : {'$substr': ['$OrderDate', 0, 7]},
        'ordercount': {'$sum': 1}
    }
}

query_2 = {
    '$sort': {'_id': 1}
}

query_3 = {
    '$project': {
        '_id' : 0,
        'Month': '$_id',
        'ordercount' : 1
    }
}

pipeline = [query_1, query_2, query_3]

r = db.Sales_Orders.aggregate(pipeline)

result = list(r)

result

[{'ordercount': 1674, 'Month': '2013-01'},
 {'ordercount': 1139, 'Month': '2013-02'},
 {'ordercount': 1683, 'Month': '2013-03'},
 {'ordercount': 1696, 'Month': '2013-04'},
 {'ordercount': 1808, 'Month': '2013-05'},
 {'ordercount': 1675, 'Month': '2013-06'},
 {'ordercount': 1886, 'Month': '2013-07'},
 {'ordercount': 1537, 'Month': '2013-08'},
 {'ordercount': 1617, 'Month': '2013-09'},
 {'ordercount': 1618, 'Month': '2013-10'},
 {'ordercount': 1552, 'Month': '2013-11'},
 {'ordercount': 1565, 'Month': '2013-12'},
 {'ordercount': 1791, 'Month': '2014-01'},
 {'ordercount': 1538, 'Month': '2014-02'},
 {'ordercount': 1586, 'Month': '2014-03'},
 {'ordercount': 1739, 'Month': '2014-04'},
 {'ordercount': 1908, 'Month': '2014-05'},
 {'ordercount': 1887, 'Month': '2014-06'},
 {'ordercount': 2008, 'Month': '2014-07'},
 {'ordercount': 1642, 'Month': '2014-08'},
 {'ordercount': 1637, 'Month': '2014-09'},
 {'ordercount': 1921, 'Month': '2014-10'},
 {'ordercount': 1628, 'Month': '2014-11'},
 {'ordercou

- Would like to know the average monthly sales prices?  

In [13]:
query_1 = {
    '$unwind' : "$OrderLines"
}

query_2 = {
    '$group': {
        '_id' : {'$substr': ['$OrderDate', 0, 7]},
        'average_monthly_sales_prices': { '$avg': { '$multiply': [ "$OrderLines.Quantity", "$OrderLines.UnitPrice" ] } }
    }
}

query_3 = {
    '$project': {
        '_id' : 0,
        'Month': '$_id',
        'average_monthly_sales_prices' : 1
    }
}

query_4 = {
    '$sort': {'Month': 1}
}

pipeline = [query_1, query_2, query_3, query_4]

r = db.Sales_Orders.aggregate(pipeline)

result = list(r)

result

[{'average_monthly_sales_prices': 724.2648835447832, 'Month': '2013-01'},
 {'average_monthly_sales_prices': 757.187922705314, 'Month': '2013-02'},
 {'average_monthly_sales_prices': 735.9580812766748, 'Month': '2013-03'},
 {'average_monthly_sales_prices': 782.0304949190817, 'Month': '2013-04'},
 {'average_monthly_sales_prices': 800.637015265836, 'Month': '2013-05'},
 {'average_monthly_sales_prices': 777.4632071937056, 'Month': '2013-06'},
 {'average_monthly_sales_prices': 763.694343622795, 'Month': '2013-07'},
 {'average_monthly_sales_prices': 747.2962440340319, 'Month': '2013-08'},
 {'average_monthly_sales_prices': 767.9943616395371, 'Month': '2013-09'},
 {'average_monthly_sales_prices': 750.3137594275769, 'Month': '2013-10'},
 {'average_monthly_sales_prices': 770.5888844058907, 'Month': '2013-11'},
 {'average_monthly_sales_prices': 750.7256141763995, 'Month': '2013-12'},
 {'average_monthly_sales_prices': 739.6302006335796, 'Month': '2014-01'},
 {'average_monthly_sales_prices': 739.698

- Would like to know the yearly expenditures with suppliers (per supplier name)?  

In [14]:
query_1 = {
    '$group': {
        '_id' : {
            'year': {'$substr': ['$TransactionDate', 0, 4]},
            'supplier': '$SupplierID'
        },
        'count' : {'$sum' : '$TransactionAmount'}
    }
}

query_2 = {
    "$lookup":
    {
       "from": "Purchasing_Suppliers",
       "localField": "_id.supplier",
       "foreignField": "SupplierID",
       "as": "supplier"
     }
}

query_3 = {
    '$project': {
        '_id' : 0,
        'year': '$_id.year',
        'supplier_name' : '$supplier.SupplierName',
        'yearly_expenditures' : '$count',                
    }
}

query_4 = {
    '$sort': {'year': 1}
}

pipeline = [query_1, query_2, query_3, query_4]

r = db.Purchasing_SupplierTransactions.aggregate(pipeline)

result = list(r)

result

[{'year': '2013',
  'supplier_name': ['Contoso, Ltd.'],
  'yearly_expenditures': 0.0},
 {'year': '2013',
  'supplier_name': ['Graphic Design Institute'],
  'yearly_expenditures': 0.0},
 {'year': '2013',
  'supplier_name': ['Litware, Inc.'],
  'yearly_expenditures': 119343.31999999998},
 {'year': '2013',
  'supplier_name': ['Northwind Electric Cars'],
  'yearly_expenditures': -3.637978807091713e-12},
 {'year': '2013',
  'supplier_name': ['The Phone Company'],
  'yearly_expenditures': 0.0},
 {'year': '2013',
  'supplier_name': ['Fabrikam, Inc.'],
  'yearly_expenditures': 485380.49999999977},
 {'year': '2014',
  'supplier_name': ['Fabrikam, Inc.'],
  'yearly_expenditures': 1510955.0999999973},
 {'year': '2014',
  'supplier_name': ['Litware, Inc.'],
  'yearly_expenditures': 686937.2100000003},
 {'year': '2015',
  'supplier_name': ['Litware, Inc.'],
  'yearly_expenditures': 1353734.6900000004},
 {'year': '2015',
  'supplier_name': ['Fabrikam, Inc.'],
  'yearly_expenditures': 2837528.3999999

Partnerships:  
- Would like to know what's the most common payment type?  


In [15]:
db.Purchasing_SupplierTransactions.create_index(
    [('PaymentMethod.PaymentMethodName', 1)],
    name='PaymentMethodName', 
)

'PaymentMethodName'

In [16]:
# We checked that all the payment methods in Purchasing_SupplierTransactions are also in Sales_CustomerTransactions so we can 
# match collections
query_1 = {
    '$group': {
        '_id' : '$PaymentMethod.PaymentMethodName', 
        'count' : {'$sum' : 1}
    }
}

query_2 = {
    "$lookup":{
        "from": "Purchasing_SupplierTransactions",
        "localField": "_id",
        "foreignField": "PaymentMethod.PaymentMethodName",
        "as": "supplier_payments"
     }
}

query_3 = {
    "$unwind": "$supplier_payments"
}

query_4 = {
    '$group': {
        '_id' : '$_id',
        'cust_count' : { '$first': '$count' },
        'supp_count' : {'$sum' : 1}
    }
}

query_5 = {
    '$project': {
        '_id' : 0,
        'payment_type' : '$_id',
        'total_count': { '$sum': [ "$cust_count", "$supp_count" ] }
    }
}

query_6 = {
    '$sort': {'year': 1}
}

limit = {'$count':1}
pipeline = [query_1, query_2, query_3, query_4, query_5, query_6]

r = db.Sales_CustomerTransactions.aggregate(pipeline)

result = list(r)

result

[{'payment_type': 'EFT', 'total_count': 54397}]

- Which supplier of `Novelty Goods Supplier` as the most transactions?  


In [18]:
db.Purchasing_SupplierTransactions.create_index(
    [('SupplierID', 1)],
    name='SupplierID', 
)

'SupplierID'

In [19]:
query_1 = {
    '$match':{
                'SupplierID':{'$in' : list(db.Purchasing_Suppliers.distinct('SupplierID',
                                    {'SupplierCategory.SupplierCategoryName':'Novelty Goods Supplier'}))
    }
}}

query_2 = {
    '$project':{
                '_id':False,
                'supplier_id':'$SupplierID'
    }
}

query_3 = {
    '$group': {
        '_id': '$supplier_id', 
        'count' : {'$sum' : 1}            
}
}

query_4 = {
    '$sort': {
        'count' : -1}            
}

query_5 =     {
        '$lookup': {
           "from": "Purchasing_Suppliers",
           "localField": "_id",
           "foreignField": "SupplierID",
           "as": "supplier_info"
        }}

query_6 = {
    '$project' :{
        '_id' : '$supplier_info.SupplierName',
        'count' : '$count'
    }
}


pipeline = [query_1, query_2, query_3, query_4, query_5, query_6]

r = db.Purchasing_SupplierTransactions.aggregate(pipeline)

result = list(r)

result

[{'_id': ['Graphic Design Institute'], 'count': 16},
 {'_id': ['The Phone Company'], 'count': 7},
 {'_id': ['A Datum Corporation'], 'count': 7},
 {'_id': ['Contoso, Ltd.'], 'count': 2}]

The marketing team:  
- Want to make an appreciation post and needs the name of the sales person with the most invoices in 2013 (person who's customers brought the most money)?


In [21]:
# Showing the sales person that generated the most value in terms of money spent by his customers
query_1 = {
    '$match': {
        "InvoiceDate": {'$regex': '.*2013.*'}
    }
}
query_2 = {
    '$unwind' : "$InvoiceLines"
}

query_3 = {
    '$group': {
        '_id' : '$SalespersonPersonID',
        'customer_money_brought': { '$sum': { '$multiply': [ "$InvoiceLines.Quantity", "$InvoiceLines.UnitPrice" ] } },
        'invoices_amount': { '$sum': 1 }
    }
}

query_4 = {
    "$lookup":
    {
       "from": "Application_People",
       "localField": "_id",
       "foreignField": "PersonID",
       "as": "person"
     }
}

query_5 = {
    '$project': {
        '_id' : 0,
        'name': '$person.FullName',
        'customer_money_brought' : '$customer_money_brought',
        'invoices_amount' : '$invoices_amount',                
    }
}

query_6 = {
    '$sort': {'customer_money_brought': 1}
}

query_7 = {
    '$limit': 1
}

pipeline = [query_1, query_2, query_3, query_4, query_5, query_6, query_7]

r = db.Sales_Invoices.aggregate(pipeline)

result = list(r)

result

[{'name': ['Taj Shand'],
  'customer_money_brought': 2113387.3,
  'invoices_amount': 2942}]